<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/NMT_Exe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) 런타임 초기화(재시작) 먼저 하는 게 깔끔합니다.
# 1) 기존 것 제거
!pip uninstall -y torch torchtext torchvision torchaudio numpy

# 2) PyTorch 1.12.1 (CUDA 11.3) 세트
!pip install --no-cache-dir \
  torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 \
  -f https://download.pytorch.org/whl/torch_stable.html

# 3) torchtext 0.13.1 (legacy 포함)
!pip install --no-cache-dir torchtext==0.13.1

# 4) NumPy는 1.x로 고정
!pip install "numpy<2"

# (선택) 쓰시던 옵티마이저
!pip install torch-optimizer

In [ ]:
import torch, torchtext, numpy
print(torch.__version__)     # 2.0.1+cu118
print(torchtext.__version__) # 0.15.2
import torchtext.legacy.data # 에러 없어야 정상

In [ ]:
!python /content/drive/MyDrive/nmt_fold/simple-nmt/train.py --train /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.bpe --valid /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.bpe --lang enko --gpu_id 0 --batch_size 160 --n_epochs 30 --max_length 64 --dropout .2 --word_vec_size 512 --hidden_size 768 --n_layers 4 --max_grad_norm 1e+8 --iteration_per_update 2 --lr 1e-3 --lr_step 0 --use_adam --model_fn /content/drive/MyDrive/nmt_fold/data1/models1.seq2seq.pth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')